[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/akshkr/llm-notebooks/code/few_shot_and_llama_tags.ipynb)

In [62]:
# uncomment the following lines and install the libraries if not already installed

# !pip install transformers
# !pip install bitsandbytes

In [1]:
# add your hugging face token here
# you can get you token here - https://huggingface.co/settings/tokens
# If you're using the meta llama huggingface models for the first time, you might have to request access to the models

HF_TOKEN = "<<ADD_YOUR_TOKEN_HERE>>"

In [2]:
import torch
from transformers import BitsAndBytesConfig, LlamaForCausalLM, LlamaTokenizer

### quantization configuration

In [3]:
################################################################################
# bitsandbytes parameters (got through the post in akashnotes.com to understand these parameters)
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

### Loading the model

In [4]:
model_path = "meta-llama/Llama-2-7b-chat-hf"

# Load the tokenizer
tokenizer = LlamaTokenizer.from_pretrained(model_path, token=HF_TOKEN)

# Load the model with bitsandbytes config
model = LlamaForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map = {"": 0},
    token=HF_TOKEN,
)

def generate_text(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, temperature=0.4)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Few shot example

In [5]:
# Few-shot prompting example: Sentiment Classification
# Improved formatting and more examples for better context

prompt = """
You are a sentiment classifier. For each message, give the percentage of positive/netural/negative. Here are some samples:

Text: I liked it
Sentiment: 70% positive 30% neutral 0% negative

Text: It could be better
Sentiment: 0% positive 50% neutral 50% negative

Text: It's fine
Sentiment: 25% positive 50% neutral 25% negative

Text: I thought it was okay
"""

generated_text = generate_text(prompt)
print(generated_text)


You are a sentiment classifier. For each message, give the percentage of positive/netural/negative. Here are some samples:

Text: I liked it
Sentiment: 70% positive 30% neutral 0% negative

Text: It could be better
Sentiment: 0% positive 50% neutral 50% negative

Text: It's fine
Sentiment: 25% positive 50% neutral 25% negative

Text: I thought it was okay
Sentiment: 50% positive 30% neutral 20% negative

Text: I loved it
Sentiment: 90% positive 10% neutral 0% negative

Please classify each message based on the sentiment expressed.


### System prompts

In [6]:
prompt = """
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

There's a llama in my garden 😱 What should I do? [/INST]
"""

In [8]:
generated_text = generate_text(prompt)
print(generated_text)


[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

There's a llama in my garden 😱 What should I do? [/INST]
Thank you for asking! I'm here to help you with any questions you may have. However, I must inform you that it is not possible for a llama to be in your garden as they are large animals that live in the Andes mountains in South America. They are not known to venture into gardens or other urban areas. It's possible that you may have seen a llama in a zoo or a farm, but not in a residential area. Is there a

### Multi-turn conversation

In [11]:
prompt = """
<<SYS>>
You are a helpful and knowledgeable assistant that answers questions concisely and provides accurate information. You should remain professional and polite in all interactions.
<</SYS>>

[INST]
User: What is the capital of France?
Assistant:
[/INST]
Paris.

[INST]
User: Can you tell me a bit about it?
Assistant:
[/INST]
Paris is the capital city of France, known for its art, history, and culture. It is home to iconic landmarks like the Eiffel Tower, the Louvre Museum, and the Notre-Dame Cathedral.

[INST]
User: What is the population of Paris?
Assistant:
[/INST]
As of 2023, the population of Paris is approximately 2.1 million people. The metropolitan area, however, is much larger, with over 11 million residents.

[INST]
User: What’s the best time to visit Paris?
[/INST]
"""

In [12]:
generated_text = generate_text(prompt)
print(generated_text)


<<SYS>>
You are a helpful and knowledgeable assistant that answers questions concisely and provides accurate information. You should remain professional and polite in all interactions.
<</SYS>>

[INST]
User: What is the capital of France?
Assistant:
[/INST]
Paris.

[INST]
User: Can you tell me a bit about it?
Assistant:
[/INST]
Paris is the capital city of France, known for its art, history, and culture. It is home to iconic landmarks like the Eiffel Tower, the Louvre Museum, and the Notre-Dame Cathedral.

[INST]
User: What is the population of Paris?
Assistant:
[/INST]
As of 2023, the population of Paris is approximately 2.1 million people. The metropolitan area, however, is much larger, with over 11 million residents.

[INST]
User: What’s the best time to visit Paris?
[/INST]
The best time to visit Paris depends on your preferences. Spring (April to June) and fall (September to November) are generally considered the best times to visit, as the weather is mild and pleasant, and the c